In [1]:
from Python.core_functions import *

import numpy as np
import pickle

from qiskit import quantum_info as qi
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error

# 1. Preparation of Random Sequences

In [2]:
# parameter definitions:
lengths = [1, 5, 10, 15, 25, 50]  # circuit lenghts 
number_qubits = 2  # number of qubits in the system
number_samples = 1000  # number of sampled sequences per length
protocol = 'multi'  # 'multi' for global Cliffords, 'local' for single-qubit Cliffords

In [13]:
# draw random sequences of Clifford gates:
sequences = draw_sequences(number_qubits, protocol, lengths, number_samples)

# construct circuits from the random sequences:
circuits = construct_circuits(sequences, number_qubits)

# construct PTMs of the Clifford gates for later post-processing:
ptms = get_ptms(sequences)

In [14]:
# optional: save the circuits and PTMs for later use and reproducibility

# save circuits:
with open('unitary_noise_circuits', 'wb') as fp:
    pickle.dump(circuits, fp)
    

# convert PTMs to sparse arrays and save:
sparse_ptms = array_to_sparse(ptms)

with open('unitary_noise_ptms', 'wb') as fp:
    pickle.dump(sparse_ptms, fp)


# if circuits and PTMs have been saved this way, this is how to retrieve them:

# with open('unitary_noise_circuits', 'rb') as fp:
#     circuits = pickle.load(fp)
    
# with open('unitary_noise_ptms', 'rb') as fp:
#     sparse_ptms = pickle.load(fp)
    
# ptms = sparse_to_array(sparse_ptms)

# 2. Insertion of Unitary Noise

In [4]:
# define the unitary noise. Here: independent ZZ rotations
def ZZ(theta1, theta2):
    
    unitary_noise = np.array([[np.exp(1j/2*(-theta2 - theta1)), 0, 0, 0],
            [0, np.exp(1j/2*(-theta2 + theta1)), 0, 0],
            [0, 0, np.exp(1j/2*(theta2 - theta1)), 0],
            [0, 0, 0, np.exp(1j/2*(theta2 + theta1))]])
    
    return unitary_noise


# choose angles:
theta1 = 0.07
theta2 = 0.13

# create a gate that can be inserted into the Qiskit circuits:
noisegate = custom_unitary(circuits[0][0], qi.Operator(ZZ(theta1, theta2)), "ZZ")

# insert the unitary noise into the circuits:
noisy_circuits = []

for length in range(len(circuits)):
    new_circs = []

    for rep in range(len(circuits[length])):
        new_circ = insert_error(circuits[length][rep], noisegate)
        new_circs.append(new_circ)

    noisy_circuits.append(new_circs)

# 3. Noisy Simulation

In [5]:
# define noise model
custom_noise = NoiseModel()
p_two = 0.01    # probability of depolarizing noise on two-qubit gates
p_single = 0.002    # probability of depolarizing noise on one-qubit gates

# choose a native gate set in which the circuits will be compiled:
native_gates = ['rx', 'ry', 'rz', 'cx']

# add noise to the gates:
custom_noise.add_all_qubit_quantum_error(depolarizing_error(p_single, 1), ['rx', 'ry', 'rz'])
custom_noise.add_all_qubit_quantum_error(depolarizing_error(p_two, 2), 'cx')

# run noisy simulation:
number_shots = 100
noisy_data = noisy_simulation(noisy_circuits, number_qubits, number_shots, number_samples, lengths, noise_model=custom_noise, gateset=native_gates)

In [6]:
# optional: save the noisy simulation output data for later use
with open('unitary_noise_output_data.npy', 'wb') as f:
    np.save(f, noisy_data)


# this is how data that was saved can be retrieved:
# with open('unitary_noise_output_data.npy', 'rb') as f:
#     noisy_data = np.load(f)

# 4. Post-processing: Unitary Noise Optimization

## 4.1 Create Plots

In [ ]:
# set N and K for the median-of-means estimation
# N*K needs to equal number_samples
N = 200
K = 5

# get explicit projection matrix. For multi-qubit Clifford protocol: only one projection, onto the traceless subspace
proj_traceless = get_projection_matrices('multi')

# set angles for the "sweep plot":
params2 = np.arange(0, 0.16, 0.01).tolist()     # angles 0, 0.01, 0.02, ..., 0.15
params1 = np.arange(0, 0.16, 0.01).tolist()

# choose a subset of angles for which the decay should be plotted as well
# (as a list of tuples giving the indices for params1 and params2)
params_to_plot = [[7, 13], [5,9], [3,5], [1,1]]

# this function calculates generalized fidelities for all combinations of parameters params1, params2,
# plots them as a heatmap, and plots individual curves of the correlators k(m) for the values given in params_to_plot
get_sweep_plot(ZZ, params1, params2, ptms, noisy_data, number_qubits, number_samples, N, K, lengths, params_to_plot, save=True)

## 4.2 Perform Nelder-Mead Optimization

In [ ]:
# define unitary noise ansatz:
def ZZ(theta1, theta2):
    
    unitary_noise = np.array([[np.exp(1j/2*(-theta2 - theta1)), 0, 0, 0],
            [0, np.exp(1j/2*(-theta2 + theta1)), 0, 0],
            [0, 0, np.exp(1j/2*(theta2 - theta1)), 0],
            [0, 0, 0, np.exp(1j/2*(theta2 + theta1))]])
    
    return unitary_noise

# median-of-means parameters:
N = 200
K = 5

# this function performs the optimization and saves all steps in "nelder_mead_path.txt"
nelder_mead_optimization(ZZ, ptms, noisy_data, number_qubits, number_samples, N, K, lengths)